In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import mmap
import json
import random
import pickle
import argparse

# Now we can use the argument values in our program
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# batch_size = args.batch_size # to use the batch_size cmd arg -> python file_name.py -batch_size 32
batch_size = 8                                            # batch size refers to the number of chunks that we process in one go
block_size = 128                                            # block size refers to the small snippets or chunks that we work on
max_iters = 6000
learning_rate = 1e-4                                        # 3e-3, 1e-3, 1e-4 
eval_iters = 100
n_embd = 384
n_head = 4
n_layer = 4
dropout = 0.2

print(device)

In [ ]:
# Load JSON data
with open('info_data.json', 'r') as f:
    data = json.load(f)


In [ ]:
# Concatenate input-output pairs into a single string
text = ''
for item in data:
    text += f"Q: {item['input']}\nA: {item['output']}\n"

chars = sorted(list(set(text)))
vocab_size = len(chars)

In [ ]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

# Encode the entire dataset
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
def get_random_chunk(data, split):
    # Determine the size of the dataset and a random position to start reading
    data_size = len(data)
    start_pos = random.randint(0, data_size - block_size*batch_size - 1)
    
    # Get a chunk of data starting from the random position
    chunk = data[start_pos:start_pos + block_size*batch_size]
    return chunk

def get_batch(data, split):
    chunk = get_random_chunk(data, split)
    ix = torch.randint(len(chunk) - block_size, (batch_size,))
    x = torch.stack([chunk[i:i+block_size] for i in ix])
    y = torch.stack([chunk[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(data, split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
class Head(nn.Module):
    """ one head of multi-head self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    print(iter,'--')
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']: .4f}, val loss: {losses['val']: .4f}")
    xb, yb = get_batch(data, 'train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('chatBot-model-01.pkl', 'wb') as f:
    pickle.dump(model, f)
print('model saved')

In [ ]:
# Load your model and vocab
with open('chatBot-model-01.pkl', 'rb') as f:
    model = pickle.load(f)
model = model.to(device)  # Move model to GPU if applicable

In [ ]:
context = torch.tensor([[string_to_int['a']]], dtype=torch.long, device=device)  # Use 'a' as the starting token
generated_tokens = model.generate(context, max_new_tokens=50)
generated_text = decode(generated_tokens.tolist()[0])

print(generated_text)

In [ ]:
import torch

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define your encode function, vocab_size, and block_size
encode = lambda s: [string_to_int.get(c, 0) for c in s]  # Handle unknown chars with 0

# Define pad_or_truncate function
def pad_or_truncate(tokens, block_size):
    if len(tokens) < block_size:
        tokens = tokens + [0] * (block_size - len(tokens))  # Pad with zeros
    else:
        tokens = tokens[:block_size]  # Truncate to block_size
    return tokens

# Define preprocess_text function
def preprocess_text(text, encode):
    tokens = encode(text)
    return torch.tensor(tokens, dtype=torch.long)

# Define prepare_input function
def prepare_input(text, encode, block_size, device):
    tokens = preprocess_text(text, encode)
    tokens = pad_or_truncate(tokens.tolist(), block_size)
    return torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension and move to device

# Load your model and vocab
with open('chatBot-model-01.pkl', 'rb') as f:
    model = pickle.load(f)
model = model.to(device)  # Move model to GPU if applicable

# Preprocess and generate text
real_data_text = "who are you"
prepared_input = prepare_input(real_data_text, encode, block_size, device)

# Generate text from the model
model.eval()  # Set model to evaluation mode
context = torch.zeros((1, 1), dtype=torch.long, device=device)  # Starting context (can be any token index, here it's 0)
generated_tokens = model.generate(context, max_new_tokens=50)
generated_text = decode(generated_tokens[0].tolist())

print(generated_text)


In [ ]:
import torch
import pickle

# Example vocabulary
vocab = [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '?']
string_to_int = {ch: i for i, ch in enumerate(vocab)}
int_to_string = {i: ch for i, ch in enumerate(vocab)}

def encode(s): 
    return [string_to_int.get(c, 0) for c in s]

def decode(tokens): 
    return ''.join(int_to_string.get(t, '') for t in tokens)

def pad_or_truncate(tokens, block_size): 
    if len(tokens) < block_size:
        tokens = tokens + [0] * (block_size - len(tokens))
    else:
        tokens = tokens[:block_size]
    return tokens

def prepare_input(text, encode, block_size, device):
    tokens = encode(text)
    tokens = pad_or_truncate(tokens, block_size)
    return torch.tensor(tokens, dtype=torch.long).unsqueeze(0).to(device)

def generate_answer(model, question, block_size, device):
    model.eval()
    prepared_input = prepare_input(question, encode, block_size, device)
    with torch.no_grad():
        generated_tokens = model.generate(prepared_input, max_new_tokens=50)
        generated_text = decode(generated_tokens[0].tolist())
    return generated_text

if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    with open('chatBot-model-01.pkl', 'rb') as f:
        model = pickle.load(f)
    model = model.to(device)

    block_size = 128
    question = "what is your name?"
    answer = generate_answer(model, question, block_size, device)
    print("Question:", question)
    print("Generated Answer:", answer)
